從《Python 網路爬蟲與資料分析入門實戰》第三章的範例中練習爬蟲

此為第三章中批踢踢八卦版爬蟲範例中練習自寫程式碼

書中原始程式碼來源：https://github.com/jwlin/web-crawler-tutorial/tree/master/ch3

# 壹、寫爬蟲函數：(1)抓取今日的文章內容(2)重複抓取動作並儲存

In [1]:
import requests
import time
import json
from bs4 import BeautifulSoup

def Gossiping_today_pop_articles(url):
    #1.與網頁溝通/連線(使用requests套件)並帶入是否滿18歲的cookies
    resp=requests.get(url,cookies={'over18': '1'})
    
    #2.剖析網頁原始碼(使用BeautifulSoup剖析器)  
    soup=BeautifulSoup(resp.text,'html5lib')
    
    
    #3.取得該網頁的內容
        #3.1取得上一頁連結：(1)先定位上一頁連結的「大概位置」於標籤名稱div/屬性名稱btn-group btn-group-paging
    paging_div = soup.find('div', 'btn-group btn-group-paging')
        
                        ##(2)再定位「細部精確」的所在位置：找到所有a標籤中第2個a的href(網址)
    PTT_URL = 'https://www.ptt.cc'        
    prev_url = PTT_URL+paging_div.find_all('a')[1]['href']
        
        #3.2取得該網頁的文章內容：(1)設置儲存文章的變數
    articles = []
        
                            ##(2)定位文章內容的區塊：標籤名稱div/屬性名稱r-ent
    divs = soup.find_all('div', 'r-ent')
        
                            ##(3)創造一個今日的日期並符合PTT日期格式
    today = time.strftime("%m/%d").lstrip('0')

                            ##(4)使用for迴圈取得文章區塊內的所需內容
    for d in divs:
        
                            ##(5)先設置條件取得符合今日的文章：如果發文日期(位置：div標籤/date屬性)是今天
        if d.find('div', 'date').text.strip()==today:  
            
                            ##(6)取得推文數並編輯：(6.1)先設置一個為0的推文數
            push_count = 0
        
                                                ##(6.2)取得字串型態的原始推文數:位置div標籤名/nrec屬性
            push_str = d.find('div', 'nrec').text
            
                                                ##(6.3)如果原始推文存在,將字串轉為數字
            if push_str:
                try:
                    push_count = int(push_str) 
                
                                                ##如果轉換失敗表示原始推文數為文字或空白('爆'或 'X1','X2')
                                                ##空白的推文數保持為0
                except ValueError:
            
                                                ##(6.4)轉換失敗的「爆」文轉成99推文數
                    if push_str == '爆':
                        push_count = 99
                        
                                                ##(6.5)轉換失敗的「X」文轉成-10的推文數
                    elif push_str.startswith('X'):
                        push_count = -10
                        
                        
                            ##(7)取得文章連結/標題/作者並存至文章的變數中
                                    ##(7.1)如果文章存在：標籤a就是文章所在位置
            if d.find('a'):
                
                                    ##(7.2)取得文章標題：標籤a的字串
                title = d.find('a').text    
                
                                    ##(7.3)取得文章日期：div標籤/date屬性
                date= d.find('div', 'date').text
                
                                    ##(7.4)取得文章作者：標籤div屬性author的字串
                author = d.find('div', 'author').text 
                
                                    ##(7.5)取得文章網址：標籤a的href
                href = PTT_URL+d.find('a')['href']   
                
                                    ##(7.5)將上述變數儲存至文章的變數中
                articles.append({
                    'title':title,
                    'date':date,
                    'author':author,
                    'push_count':push_count,
                    'href':href})
                
                        ##(8)將程式結果返回文章內容
    return articles,prev_url


#4.取得該網頁的上一頁的文章內容(重複1.2.3的動作)
def repeat_articles(url):
    
    ##(4.1)創造放文章的變數
    articles=[]
    
    ##(4.2)利用自寫函數取得「原網頁文章內容」與「上一頁連結」
    new_articles,prev_url=Gossiping_today_pop_articles(url)
    
    ##(4.3)將原網頁內容放入變數中
    articles += new_articles
    
    ##(4.4)利用字寫函數取得的「上一頁連結」來取得「上一頁的文章內容與連結」
    new_articles, new_prev_url= Gossiping_today_pop_articles(prev_url)
    
    ##(4.5)如果「上一頁內容」不是空的=(文章日期為今日)
        ###因Gossiping_today_pop_articles函數中設定只抓取今日文章,若非今日變數則為[]
    while new_articles!=[]:
        
        ##(4.5.1)將「上一頁文章」放入變數中
        articles += new_articles
        
        ##(4.5.2)創造更新的「上一頁文章與連結」
        new_articles,new_prev_url=Gossiping_today_pop_articles(new_prev_url)
        
    ##(4.6)如果新的「上一頁內容」是空的=(文章日期非今日)則印出上述合併的所有文章內容
    else:
        return articles

# 貳、處理資料：(1)印出>50的今日文章 (2)所有今日文章存成csv檔案

In [2]:
#1.印出今日推文數>50的文章
    ##1.1將自寫函數設為文章變數
url='https://www.ptt.cc/bbs/Gossiping/index.html'
articles=repeat_articles(url)

    ##1.2印出總文章數
print('今天有', len(articles), '篇文章')

    ##1.3印出推文數>50的文章
print('熱門文章(> 50 推):' )    
for a in articles:
    if int(a['push_count']) > 50:
        print(a)

今天有 929 篇文章
熱門文章(> 50 推):
{'title': '[爆卦] 民進黨也提逕付二讀案啦！', 'date': '11/15', 'author': 'EvilisGood', 'push_count': 98, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1573803527.A.22B.html'}
{'title': '[新聞] 韓國瑜最沉重時曾同時背3間 每月房貸約24萬', 'date': '11/15', 'author': 'kid33', 'push_count': 55, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1573803738.A.FA0.html'}
{'title': 'Re: [爆卦] 今日被民進黨封殺之法案', 'date': '11/15', 'author': 'askaleroux', 'push_count': 99, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1573802102.A.A16.html'}
{'title': '[新聞] 全台獨家！新竹推超狂「巨無霸桶裝珍奶」', 'date': '11/15', 'author': 'str8g', 'push_count': 66, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1573802242.A.8F2.html'}
{'title': '[新聞] 最狂庶民！不含海外資產\u3000韓國瑜六地炒', 'date': '11/15', 'author': 'a828203', 'push_count': 66, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1573802339.A.BC8.html'}
{'title': '[新聞] 不斷更新 國民黨不分區名單曝光 謝龍介越', 'date': '11/15', 'author': 'yor', 'push_count': 86, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1573801004.A.0B7.html'}
{'title

In [3]:
import pandas as pd
import numpy as np

#2.儲存所有今日文章
    ##2.1將爬下來的文章設為字典(dict)
dict=repeat_articles(url)

    ##2.2將字典轉成DataFrame型式(columns可不寫入，資料中列的名稱會照字母排序)
name=['title','date','author','push_count','href']
test=pd.DataFrame(columns=name,data=dict)

    ##2.3把索引改成從1開始
test.index = np.arange(1,len(test)+1)
    
    ##2.4將索引名稱改為編號
test.index.names = ['article_NO.']

    ##印出結果並儲存成CSV檔案
print(test)
test.to_csv('testcsv.csv',encoding='utf_8_sig')

                                    title   date        author  push_count  \
article_NO.                                                                  
1                       [問卦] 智力測驗後發現自己是智障  11/15      hiimlove           4   
2                    Re: [爆卦] 今日被民進黨封殺之法案  11/15        takase           0   
3                          [問卦] 有沒有庶民投資方式  11/15         werqq           1   
4                    Re: [爆卦] 今日被民進黨封殺之法案  11/15       newsyho           1   
5            Re: [新聞] 國民黨最新不分區出爐 曾銘宗第一、張顯  11/15      DreamYeh           0   
6                 [新聞] 狼軍人性慾不斷！硬上前女友後速撿屍夜  11/15         RRADA           8   
7                  [問卦] 唱衰台灣棒球弱的日本高中生臉腫了嗎  11/15       sulanpa           0   
8               [新聞] 12強胡金龍陽春砲 中華隊2比1領先美國  11/15  andandandand           0   
9                  [問卦] 欸！星巴克買一送一怎麼點比較聰明？  11/15  sean12345678           3   
10               [新聞] 引爆直播主之亂！連千毅求回家顧小孩 仍  11/15    kawazakiz2           6   
11                  Re: [問卦] 智力測驗後發現自己是智障  11/15   liang691206  

# 無註解程式碼

In [4]:
import requests
import time
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np



def Gossiping_today_pop_articles(url):
    resp=requests.get(url,cookies={'over18': '1'})
    soup=BeautifulSoup(resp.text,'html5lib')
    
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    PTT_URL = 'https://www.ptt.cc'        
    prev_url = PTT_URL+paging_div.find_all('a')[1]['href']
   

    articles = []
    divs = soup.find_all('div', 'r-ent')

    today = time.strftime("%m/%d").lstrip('0')

    
    for d in divs:
        if d.find('div', 'date').text.strip()==today:  
            push_count = 0
            push_str = d.find('div', 'nrec').text
            
            if push_str:
                try:
                    push_count = int(push_str) 
                except ValueError:
                    if push_str == '爆':
                        push_count = 99
                    elif push_str.startswith('X'):
                        push_count = -10

            if d.find('a'):
                title = d.find('a').text    
                date= d.find('div', 'date').text
                author = d.find('div', 'author').text 
                href = PTT_URL+d.find('a')['href']   
                articles.append({
                    'title':title,
                    'date':date,
                    'author':author,
                    'push_count':push_count,
                    'href':href})

    return articles,prev_url



def repeat_articles(url):
    articles=[]
    
    new_articles,prev_url=Gossiping_today_pop_articles(url)
    articles += new_articles
    new_articles, new_prev_url= Gossiping_today_pop_articles(prev_url)

    while new_articles!=[]:
        articles += new_articles
        new_articles,new_prev_url=Gossiping_today_pop_articles(new_prev_url)

    else:
        return articles

    
    
    
    
    
    
url='https://www.ptt.cc/bbs/Gossiping/index.html'    
articles=repeat_articles(url)

print('今天有', len(articles), '篇文章')
print('熱門文章(> 50 推):' )    
for a in articles:
    if int(a['push_count']) > 50:
        print(a)


        
        
dict=repeat_articles(url)
name=['title','date','author','push_count','href']
test=pd.DataFrame(columns=name,data=dict)
test.index = np.arange(1,len(test)+1)
test.index.names = ['article_NO.']


print(test)
test.to_csv('testcsv.csv',encoding='utf_8_sig')

今天有 931 篇文章
熱門文章(> 50 推):
{'title': '[爆卦] 民進黨也提逕付二讀案啦！', 'date': '11/15', 'author': 'EvilisGood', 'push_count': 99, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1573803527.A.22B.html'}
{'title': '[新聞] 韓國瑜最沉重時曾同時背3間 每月房貸約24萬', 'date': '11/15', 'author': 'kid33', 'push_count': 55, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1573803738.A.FA0.html'}
{'title': 'Re: [爆卦] 今日被民進黨封殺之法案', 'date': '11/15', 'author': 'askaleroux', 'push_count': 99, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1573802102.A.A16.html'}
{'title': '[新聞] 全台獨家！新竹推超狂「巨無霸桶裝珍奶」', 'date': '11/15', 'author': 'str8g', 'push_count': 67, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1573802242.A.8F2.html'}
{'title': '[新聞] 最狂庶民！不含海外資產\u3000韓國瑜六地炒', 'date': '11/15', 'author': 'a828203', 'push_count': 66, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1573802339.A.BC8.html'}
{'title': '[新聞] 不斷更新 國民黨不分區名單曝光 謝龍介越', 'date': '11/15', 'author': 'yor', 'push_count': 86, 'href': 'https://www.ptt.cc/bbs/Gossiping/M.1573801004.A.0B7.html'}
{'title

                                    title   date        author  push_count  \
article_NO.                                                                  
1                       [問卦] 智力測驗後發現自己是智障  11/15      hiimlove           4   
2                    Re: [爆卦] 今日被民進黨封殺之法案  11/15        takase           0   
3                          [問卦] 有沒有庶民投資方式  11/15         werqq           1   
4                    Re: [爆卦] 今日被民進黨封殺之法案  11/15       newsyho           1   
5            Re: [新聞] 國民黨最新不分區出爐 曾銘宗第一、張顯  11/15      DreamYeh           0   
6                 [新聞] 狼軍人性慾不斷！硬上前女友後速撿屍夜  11/15         RRADA           9   
7                  [問卦] 唱衰台灣棒球弱的日本高中生臉腫了嗎  11/15       sulanpa           0   
8               [新聞] 12強胡金龍陽春砲 中華隊2比1領先美國  11/15  andandandand           1   
9                  [問卦] 欸！星巴克買一送一怎麼點比較聰明？  11/15  sean12345678           3   
10               [新聞] 引爆直播主之亂！連千毅求回家顧小孩 仍  11/15    kawazakiz2           6   
11                  Re: [問卦] 智力測驗後發現自己是智障  11/15   liang691206  